# Programming for Data Science and Artificial Intelligence

## Supervised Learning - Classification - Logistic Regression - Binary

### Readings: 
- [GERON] Ch4
- [VANDER] Ch5
- [HASTIE] Ch4

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Logistic Regression

Logistic regression is an supervised algorithm for solving classification problem where outcome (target) is discrete.   The idea behind is that $\boldsymbol{\theta}^T\mathbf{x}$ will return a continous value and thus may not be suitable for classification task.  However, if we can find one function $g$ such that

$$g(\boldsymbol{\theta}^T\mathbf{x}) = [0, 1]$$

then we can define our hypothesis function as $g$ and optimize accordingly based on some cost function.

It happens that $g$ (and also our hypothesis function $\mathbf{h}$) can be defined as the sigmoid (logit) function as the following:

$$ h = g(\boldsymbol{\theta}^T\mathbf{x}) = \frac{1}{1+e^{-\boldsymbol{\theta}^T\mathbf{x}}} $$

**Trivials**: 
$e$ is a really convenient number for math, for example whenever you take the derivative of $e^x$, you get $e^x$ back again.  It's the only function on Earth that will do that.  Also, $e^x$ always give you positive numbers, thus it is no surprise this $e$ was often used in probability/statistics.  Last, it is convenient to apply $\log$ in any optimization problem including $e$ since it will cancel it nicely and will also not change the optimization answer since $\log$ is monotically increasing.  Btw, the common log base we use in natural log, but it really does not matter because the base is merely a constant.

Let's see how does it look in code:

Recall the derivative using quotient rule is        

$$ (\frac{f}{g})' = \frac{f'g - fg'}{g^2}$$

Given sigmoid function as

$$ g(x) = \frac{1}{1+e^{-x}} $$

Thus the derivative of sigmoid function is

$$
\begin{aligned}
    \frac{dg}{dx} &= \frac{0(1 + e^{-x}) - (-1)(e^{-x}))}{(1 + e^{-x})^2} \\
    &= \frac{e^{-x}}{(1 + e^{-x})^2}  = \frac{e^{-x} + 1 - 1}{(1 + e^{-x})^2} \\
    &= \frac{1}{(1 + e^{-x})} - \frac{1}{(1 + e^{-x})^2} \\
    &= \frac{1}{(1 + e^{-x})} \big(1 - \frac{1}{(1 + e^{-x})}\big)\\
    &= g(1 - g)
\end{aligned}
$$

Let's look at the gradient by modifying our sigmoid function a little bit.

**Trivials**: As you can see, the greatest gradient possible is 0.25.  Thus it means that in a gradient descent update, the update speed will be restricted by this value.

So if we had the following function

$$ \boldsymbol{\theta}^T\mathbf{x} = \theta_{0} + \theta_{1}x_{1} + \theta_{2}x_{2} $$

Given some point ($x_{1}$,$x_{2}$), if we plugged it to our sigmoid function, the equation could output a positive result (for one class), negative result (for the other class), or 0 (the point lies right on the decision boundary).

For example, given ($x_{1} = 3$, $x_{2} = 4$) and ($\theta_{1}$ = 1, $\theta_{2}$ = 2) (let's ignore $\theta_{0}$ for simplicity), the following code performs a sigmoid of $\boldsymbol{\theta}^T\mathbf{x}$

### Where are we so far?

Well, we have so far motivated the followings:

1. We need a squashing function $g$ to use in classification problem
2. We will also go through together different possible squashing function including sigmoid, tanh, relu, and leaky relu

For now, we shall use only sigmoid function, since we will be talking mainly about logistic regression.  But we shall go back to these other activation functions later on in the course.

Here, we shall explore three different variants of logistic regression:
1. Binary Logistic Regression
2. Multinomial (multiclass) Logistic Regression
3. Logistic Regression with Newton-Raphson method

Last, we shall explore the sklearn way.

**Be warned**: There will be a lot of equations but they are necessary to understand in order to do the implementation.  For some obvious derivations, I will leave them as your exercise, but if you feel inimidated, ask me in class or come to my office.

## 1. Binary Logistic Regression

Logistic regression is a binary classification algorithm by simply finding a best fitted line that separates two dataset.  In order to squash the output to a value between 0 and 1, logistic regression used a function called logit function (or sigmoid function)

### Scratch

**Implementation steps:**
    
1. Prepare your data
    - add intercept
    - $\mathbf{X}$ and $\mathbf{y}$ and $\mathbf{w}$ in the right shape
        - $\mathbf{X}$ -> $(m, n)$
        - $\mathbf{y}$ -> $(m, )$
        - $\mathbf{w}$ -> $(n, )$
        - where $m$ is number of samples
        - where $n$ is number of features
    - train-test split
    - feature scale
    - clean out any missing data
    - (optional) feature engineering
2. Predict and calculate the loss
    - The loss function is the *cross entropy* defined as
    $$J = - (\sum_{i=1}^m y^{(i)}\log(h^{(i)}) + (1-y^{(i)})\log(1-h^{(i)}))$$
    where h is defined as the sigmoid function as
    $$h = \frac{1}{1+e^{-\boldsymbol{\theta}^T\mathbf{x}}}$$
3. Calculate the gradient based on the loss
    - The gradient of $\theta_j$ is defined as
        $$\frac{\partial J}{\partial \theta_j} = \sum_{i=1}^{m}(h^{(i)}-y^{(i)})x_j$$
    - This can be derived by knowing that 
        $$J= - (y \log h + (1 - y) \log (1-h))$$
        $$h = \frac{1}{1+e^{-g}}$$
        $$g = \theta^Tx$$
    - Thus, gradient of $J$ in respect to some $\theta_j$ is
        $$\frac{\partial J}{\partial \theta_j} = \frac{\partial J}{\partial h} \frac{\partial h}{\partial g} \frac{\partial g}{\partial \theta_j}$$
      where
        $$\frac{\partial J}{\partial h} = \frac{h - y}{h(1-h)}$$
        $$\frac{\partial h}{\partial g} = h(1-h)$$
        $$\frac{\partial g}{\partial \theta_j} = x_j$$
    - Thus, 
    $$
    \begin{aligned}
    \frac{\partial J}{\partial \theta_j} &= \frac{\partial J}{\partial h} \frac{\partial h}{\partial g} \frac{\partial g}{\partial \theta_j}\\
   &= \frac{h - y}{h(1-h)} * h(1-h) * x_j\\
   &= (h - y)x_j\\
   \end{aligned}$$
4. Update the theta with this update rule
    $$\theta_j := \theta_j - \alpha * \frac{\partial J}{\partial \theta_j}$$
    where $\alpha$ is a typical learning rate range between 0 and 1
5. Loop 2-4 until max_iter is reached, or the difference between old loss and new loss are smaller than some predefined threshold tolerance

#### Step 1: Prepare your data

#### 1.1 Get your X and y in the right shape

#### 1.2 Feature scale your data to reach faster convergence

#### 1.3 Train test split your data

#### 1.4 Add intercepts

#### Step 2: Fit your algorithm 

#### 1. Define your algorithm

#### 2. Compute accuracy

### Classification metrics

Let us study some classification metrics that are quite different from the $r^2$ or $mse$ that we see from the regression.  Let me define a confusion matrix that looks like this:

<code>
		 	    Actual
			    +	   -
Predicted +     TP     FP
          -     FN     TN
</code>

TP is defined as true positives, FP as false positives, FN as false negatives, and TN as true negatives.

#### Accuracy, Recall, Precision, F1

Accuracy is straightforward

$$ Accuracy = \frac{TP + TN}{TP + TN + FP + FN} $$

Accuracy is mostly avoided, unless your model is really balanced of both positives and negatives.  Instead, more useful classification metrics would be precision, recall, and f1-score

$$ Precision = TP / (TP + FP) $$

Precision is useful as metric when you want to prioritize removing false positive.  Example is search engine in which you do not want to return any search results that are "false positive"

$$ Recall = TP / (TP + FN) $$

Recall is useful as metric when you want to prioritize removing false negative.  Example is cancer detection in which you do not want to miss detecting any real positive (i.e., false negative).

$$ F1 = 2 x \frac{Precision * Recall}{Precision+Recall} $$

F1 is simply seeking a balance between Precision and Recall.  Also F1 is good metric when there is an uneven class distribution (large number of actual negatives)

To get accuracy, recall, precision and f1 score, we can use **sklearn.metrics.classification_report**.

#### ROC

An ROC curve shows the performance of one classification model at **all classification thresholds**. For example, if we set threshold to 0.4, then anything less than 0.4 will be negative class, and otherwise positive class.  To build the ROC curve, you iterate all possible threshold, and collect the TP, FP, TN, TP of all possible threshold.

ROC curves typically feature true positive rate on the Y axis, and false positive rate on the X axis, where

$$ TPR = TP / (TP + FN) $$

$$ FPR = FP / (FP + TN) $$

This means that the top left corner of the plot is the “ideal” point - a false positive rate of zero, and a true positive rate of one. This is not very realistic, but it does mean that a larger area under the curve (AUC) is usually better.

To get area score under the curve, we can use **sklearn.metrics.roc_auc_score**

#### Precision-Recall

Davis and Goadrich in this paper (https://ftp.cs.wisc.edu/machine-learning/shavlik-group/davis.icml06.pdf) propose that Precision-Recall (PR) metric will be more informative than ROC when dealing with highly skewed datasets. Because Precision is directly influenced by class imbalance so the Precision-recall are better to highlight differences between models for highly imbalanced data sets. When you compare different models with imbalanced settings, the area under the Precision-Recall curve will be more sensitive than the area under the ROC curve.

Example of drawback of ROC curve

- $TPR = TP / (TP + FN) $
- $FPR = FP / (FP + TN) $

======balanced data=====
- n_sample = 500
- pos = 250
- neg = 250
- Given the following confusion matrix

<code>
			Actual
			  +	     -
Predict.+     125     125
        -     0       250
</code>

- $TPR = 125 / (125 + 0) = 1$
- $FPR = 125 / 125 + 250 = 0.3$

Looks ok!

=====imbalanced data=======
- n_sample = 500
- pos = 30
- neg = 470
- Given the following confusion matrix

<code>
			Actual
			  +	     -
Predict.+     15     15
        -     0      470
</code>

- $TPR = 15 / (15 + 0)$ = 1
- $FPR = 15/ (15 + 470) {\approx} 0$

Perfect model??  How?  Because the amount of wrong positives is undermined by the great amount of negatives

====Precision-Recall curve works much better for imbalanced=====

- Precision =  TP / (TP + FP)
- Recall = TP / (TP + FN)

- Precision = 15 / (15 + 15) = 0.5. #minimize false positive
- Recall = 15 / (15 + 0) = 1.   #minimize false negative

Reflect much better!

For precision-recall metric, we can use **sklearn.metrics.average_precision_score** which compute the ratio between recall and precision (read more here -->https://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html#sphx-glr-auto-examples-model-selection-plot-precision-recall-py). 

**Note**: sklearn version of average_precision_score are typically used in binary classification to study the output of a classifier. In order to extend the precision-recall curve and average precision to multi-class or multi-label classification, it is necessary to **binarize** the output. I have demonstrated this at the multinomial logistic regression part

### Sklearn

### === Task ===

1. Put everything into a class called LogisticRegression. The training method should be "minibatch".

2. Perform a classification on the data given above.

3. Plot training losses as number of iters increases.

4. Write a class called classification_report containing 4 functions (Accuracy, Recall, Precision, F1) and use it to evaluate your model.